In [1]:

from library.all_experiments import *  # experiments are defined here
from library.all_imports import *

/home/katta/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:455: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/home/katta/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:456: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/home/katta/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.py:457: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/home/katta/anaconda3/lib/python3.6/site-packages/tensorflow/python/framework/dtypes.

Using TensorFlow version 1.0.1


/home/katta/anaconda3/lib/python3.6/site-packages/dask/dataframe/utils.py:14: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm
/home/katta/anaconda3/lib/python3.6/site-packages/sklearn/utils/deprecation.py:143: FutureWarning: The sklearn.datasets.samples_generator module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.datasets. Anything that cannot be imported from sklearn.datasets is now part of the private API.
  warnings.warn(message, FutureWarning)


In [2]:
import os

dataset = "apache"
dataset_number = 18
epochs_ = 2
experiment_type = 'biLSTM_encoder'
experiment_sub_dir = experiment_type +"_" + str(epochs_)
results_ = "results/" + experiment_sub_dir

#create result sub directory if not exists
if not os.path.exists(results_):
    os.mkdir(results_)
    print(results_, "directory was created")

from library.all_imports import *
from library.helpers import save_to_json, load_from_json, save_to_csv, create_if_not_exists, get_N_HexCol, multiprocess_file
import pandas as pd
from sklearn.cluster import Birch
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
import seaborn as sns

sns.set()
from sklearn.datasets.samples_generator import make_blobs
# from sklearn.cluster import Birch
from sklearn.metrics import homogeneity_completeness_v_measure, silhouette_score, mutual_info_score, \
    adjusted_mutual_info_score
import re
import sys
from collections import defaultdict
import scipy.special
from itertools import combinations

assert tf.__version__ == "1.0.1"  # the tensorflow library we used
tf.set_random_seed(0)
np.random.seed(0)  # fix random seed for reproducability
tf.logging.set_verbosity(tf.logging.INFO)

"""# 0 Hyperparameter"""

'''
def lib_name(exp_nr):
    if exp_nr in EXPERIMENT_LIB_MAPPING:
        return "data_generation.%s" % EXPERIMENT_LIB_MAPPING[exp_nr]
    else:
        return "data_generation.%s" % ALL_EXPERIMENTS[exp_nr]
'''

experiment_nr = dataset_number
EXPERIMENT_ID = ALL_EXPERIMENTS[experiment_nr]  # choose data - it will be automatically generated
print("Running experiment: %s" % EXPERIMENT_ID)
EXPERIMENT_SPLIT_TOKEN = SPLIT_TOKEN[experiment_nr] if (experiment_nr in SPLIT_TOKEN.keys()) else SPLIT_TOKEN["default"]

unknown_token = g.unknown_token = "UNKNOWN_TOKEN"
logline_start_token = g.logline_start_token = "LOG_START"
logline_end_token = g.logline_end_token = "LOG_END"
pad_token = g.pad_token = "PAD_TOKEN"
g.vocabulary_max_lines = -1  # -1 means unlimited
g.max_line_len = 200

# define names
run_tag = "m-pn"
model_name = "rnn-autoencoder"

# clustering
cluster_alg = "birch"  # alternatives:  birch hierarch dbscan

# create experiment directories
result_dir = results_
experiment_dir = "%.2d_%s" % (experiment_nr, EXPERIMENT_ID)
experiment_outdir = join_path(result_dir, experiment_dir, experiment_sub_dir)
# checkpoint_path = join_path(experiment_dir, "graph", run_tag)
# graph_dir = join_path(experiment_outdir, "graph", run_tag)
graph_dir = join_path(experiment_outdir, "graph", run_tag)
#
# create_if_not_exists(checkpoint_path)
create_if_not_exists(graph_dir)

# data files

datafile = g.datafile = "data/%s.log" % EXPERIMENT_ID
labels_true_file = "data/%s.ids" % EXPERIMENT_ID
processed_datafile = join_path(result_dir, experiment_dir, "%s_log.processed" % EXPERIMENT_ID)

test_data_processed = join_path(result_dir, experiment_dir, "%s_test_log.processed" % EXPERIMENT_ID)

VOCABULARY_FILE = g.VOCABULARY_FILE = join_path(result_dir, experiment_dir, "vocabulary.json")
HYPERPARAMETERS_FILE = join_path(experiment_outdir, "hyperparams.json")
CLUSTERING_RESULTS_FILE = join_path(experiment_outdir, "clustering_results.csv")
learned_weights_file = join_path(experiment_outdir, "learned_weights.csv")
DATASET_STATS_FILE = join_path(result_dir, experiment_dir, "dataset_statistics.json")

g.REGENERATE_VOCABULARY_FILES = True
g.REPROCESS_TRAININGS_DATA = True
learn_model = True

"""# Generate vocabulary

To generate our vocabulary, we perform the following steps:
* tokenize the log lines of the data file
* create Vocabulary from tokenized loglines, count word frequencies
* create index_to_word and word_to_index dictionary
* replace unknown words with an UNK token (not relevant for us now)

If you need to see which steps are taken, have a look at  library.vocabulary.py
"""

g.WORD_TO_INDEX_FILE = join_path(result_dir, experiment_dir, "word_to_index.json")
g.INDEX_TO_WORD_FILE = join_path(result_dir, experiment_dir, "index_to_word.json")
g.TOKENIZED_LOGLINES_FILE = join_path(result_dir, experiment_dir, "tokenized_loglines.json")
g.SPLIT_TOKEN = EXPERIMENT_SPLIT_TOKEN

# g.SPLIT_TOKEN = ['"']
from library.vocabulary import *

print("Dataset fully loaded")

'''
"""# Datset statistics"""

# for w in sorted(word_frequencies.iteritems(), key= lambda x:x[1], reverse=True):
del word_frequencies["LOG_START"]
del word_frequencies["LOG_END"]
#    print(w)

words, frequencies = zip(*sorted(word_frequencies.items(), key=lambda x: x[1], reverse=True))
print("sorted words")
print(words[0:200])

total_words = len(words)
stratify_parts = 200

stratum_length = total_words / stratify_parts

indices = []

for i in range(stratify_parts):
    strat_start = i * stratum_length
    strat_end = (i + 1) * stratum_length - 1
    new_index = np.random.randint(strat_start, strat_end)
    #    print(strat_start, strat_end, new_index)
    indices.append(new_index)

words = np.array(words)[indices]
frequencies = np.array(frequencies)[indices]

print("Sorted words.")

words = [str(w) for w in words]
colors = []
for w in words:
    if w.isdigit():
        colors.append("r")  # "#ffcccc" light red
    else:
        colors.append("#ccffcc")  # light green
print("Defined colors")

cluster_plot_file = join_path(result_dir, experiment_dir, "dataset_stats.png")
figure_size = (20, 10)
fig = plt.figure(figsize=figure_size)

index = np.arange(len(words))
bar_width = 0.35

ax1 = fig.add_subplot(111)
ax1.bar(index, frequencies, color=colors)
ax1.set_yscale('log')
plt.xticks(index + bar_width / 2, (words), rotation=90, fontsize=8)
plt.savefig(cluster_plot_file)
plt.close()

# Image(filename=cluster_plot_file)

"""# Average LogLine Length"""

line_lengths = []
for logline in tokenized_loglines:
    line_lengths.append(len(logline))

np.average(line_lengths)
'''

"""# 1.3 Process trainingsdata"""

# Create the training data
if g.REPROCESS_TRAININGS_DATA:
    X_train = np.asarray([[word_to_index[w] for w in logline] for logline in tokenized_loglines])
    maximum_sequence_length = -1
    train_numbers_file = open(processed_datafile, "w")
    one_percent = len(X_train) / 100
    for i, logline_as_word_id_sequence in enumerate(X_train):
        if i % one_percent == 0: print("Written line %i" % i)
        # print ",".join(line)

        reversed_seq = list(reversed(logline_as_word_id_sequence))
        # add sequence length to example

        sequence_length = str(len(logline_as_word_id_sequence))
        if len(logline_as_word_id_sequence) > maximum_sequence_length: maximum_sequence_length = len(
            logline_as_word_id_sequence)  # find maximum sequence length
        word_id_seq = ",".join(map(str, logline_as_word_id_sequence))  # encoder input: 1,2,3
        word_id_seq_reversed = ",".join(map(str, reversed_seq))  # decoder input: 3,2,1
        target_seq = ",".join(map(str, reversed_seq[1:] + [PAD_ID]))  # decoder target: 2,1,PAD

        signature_id = 5  # experiment_lib.extract_pattern_id(loglines[i]) -------- not relevent---------------
        assert not signature_id == 0, "Each log line has to be associated to one signature, none for (%s). Check extract_pattern_id method." % \
                                      loglines[i]
        # write
        train_numbers_file.write(
            "%s|%s|%s|%s|%s\n" % (signature_id, sequence_length, word_id_seq, word_id_seq_reversed, target_seq))

    train_numbers_file.close()

    ##############

    #### tokenizing test data part
    tokenized_test_loglines = []

    test_loglines = list(open("data/test/%s.log" % EXPERIMENT_ID, 'r'))
    print("Loaded %i test_loglines" % len(test_loglines))
    total_lines = len(test_loglines) if g.vocabulary_max_lines == -1 else g.vocabulary_max_lines
    print("Tokenizing test %i lines... " % total_lines)
    print("test_loglines", len(test_loglines))

    for i, test_logline in enumerate(test_loglines):

        test_logline = test_logline.lower()

        for char in g.SPLIT_TOKEN:
            test_logline = test_logline.replace(char, ' ' + char + ' ')

        tokenized_test_logline = test_logline.split(" ")[0:200]  # nltk.word_tokenize(logline)

        tokenized_test_logline = [g.logline_start_token] + tokenized_test_logline + [g.logline_end_token]

        tokenized_test_loglines.append(tokenized_test_logline)

        if g.vocabulary_max_lines > 0 and i > g.vocabulary_max_lines:
            break

    for i, test_logline in enumerate(tokenized_test_loglines):
        tokenized_test_loglines[i] = [w if w in word_to_index else g.unknown_token for w in test_logline]

    # for i in tokenized_test_loglines:
    #     print(i)
    print("Tokenized logfile.")
    #### tokeninzing test data part finish

    array_to_xtrain = []

    X_test = np.asarray([[word_to_index[w] for w in logline] for logline in tokenized_test_loglines])
    ### put UNK into words in test that were not in train
    # for logline in tokenized_test_loglines:
    #   numbered_line = []
    #   for w in logline:
    #     if w in word_to_index:
    #       numbered_line.append(word_to_index[w])
    #     else:
    #       numbered_line.append(word_to_index["UNK_TOKEN"])
    #   array_to_xtrain.append(numbered_line)
    # X_test = np.asarray(array_to_xtrain)
    #### put UNK into words in test that were not in train FINISHED

    maximum_sequence_length = -1
    test_numbers_file = open(test_data_processed, "w")
    one_percent = len(X_test) / 100

    for i, test_logline_as_word_id_sequence in enumerate(X_test):
        if i % one_percent == 0: print("Written line %i" % i)
        # print ",".join(line)

        reversed_seq = list(reversed(test_logline_as_word_id_sequence))
        # add sequence length to example

        sequence_length = str(len(test_logline_as_word_id_sequence))
        if len(test_logline_as_word_id_sequence) > maximum_sequence_length: maximum_sequence_length = len(
            test_logline_as_word_id_sequence)  # find maximum sequence length
        word_id_seq = ",".join(map(str, test_logline_as_word_id_sequence))  # encoder input: 1,2,3
        word_id_seq_reversed = ",".join(map(str, reversed_seq))  # decoder input: 3,2,1
        target_seq = ",".join(map(str, reversed_seq[1:] + [PAD_ID]))  # decoder target: 2,1,PAD

        # signature_id = experiment_lib.extract_pattern_id(loglines[i]) #######
        signature_id = 5

        assert not signature_id == 0, "Each log line has to be associated to one signature, none for (%s). Check extract_pattern_id method." % \
                                      loglines[i]
        # write
        test_numbers_file.write(
            "%s|%s|%s|%s|%s\n" % (signature_id, sequence_length, word_id_seq, word_id_seq_reversed, target_seq))
    test_numbers_file.close()
    ##############
else:
    X_train = np.asarray([[word_to_index[w] for w in logline] for logline in tokenized_loglines])
    print("Trainingsdata has already been processed")

if not os.path.exists(labels_true_file):
    os.system("python3 create_true_labels.py -en %s" % EXPERIMENT_ID)

"""# 2.0  Graph Hyper parameters"""

# TDB save vocabulary file
state_size = 256  # size of vector internal of a cell
batch_size = 200

num_examples_to_visualize = min(10000, len(tokenized_loglines))  # how many dots to show
num_examples_to_embed = len(
    tokenized_test_loglines)  # len(tokenized_loglines) -------------changed according to out test set------------

dropout_keep_probability = 0.7
num_lstm_layers = 1

DTYPE = tf.float32
num_samples = min(vocabulary_size, 500)  # number of samples to draw for sampled softmax
max_gradient_norm = 0.5  # to be defined

LEARNING_RATE = 0.02
learning_rate_decay_factor = 0.95
l1_scale = 0.000
epochs = epochs_

num_examples = len(X_train)
max_steps = int(
    epochs * (num_examples / batch_size))  # 10*11000/200 for unix log ---------------commwnted----------------
learning_rate_adjustments = 10  # how many times to apply learning rate decay over all epochs
adjust_learning_rate_after_steps = max_steps / learning_rate_adjustments

# some tf varioables
tf_keep_probabiltiy = tf.constant(dropout_keep_probability)
tf_global_step = tf.Variable(0, trainable=False)

# clustering hierarchy
examples_in_hierarchy = 50
color_threshhold = 0.2

# threshold for vmeasure, homogenity etc.
h_threshold = 0.004

# save hyperparameters
hyperparams = {
    "state_size": state_size,
    "num_examples_to_visualize": num_examples_to_visualize,
    "dropout_keep_probability": dropout_keep_probability,
    "num_lstm_layers": num_lstm_layers,
    "learning_rate_decay_factor": learning_rate_decay_factor,
    "batch_size": batch_size,
    "dtype": str(DTYPE),
    "num_samples": num_samples,
    "max_gradient_norm": max_gradient_norm,
    "learning_rate": LEARNING_RATE,
    "epochs": epochs,
    "num_examples": num_examples,
    "max_steps": max_steps,
    "examples_in_hierarchy": examples_in_hierarchy,
    "color_threshhold": color_threshhold,
    "h_threshold": h_threshold,
}

save_to_json(hyperparams, HYPERPARAMETERS_FILE)

"""#  2 Graph

## Learning Rate
"""

# https://www.tensorflow.org/versions/master/api_docs/python/tf/train/exponential_decay

learning_rate = tf.train.exponential_decay(
    learning_rate=LEARNING_RATE,
    global_step=tf_global_step,  # current learning step
    decay_steps=adjust_learning_rate_after_steps,  # how many steps to train after decaying learning rate
    decay_rate=learning_rate_decay_factor,
    staircase=True)

"""## 2.1 Input, Output and Target of the graph"""

# inputs, outputs
x_e = tf.placeholder(tf.int32, [batch_size, None])  # encoder inputs loglines [batch_size, num_steps]
x_d = tf.placeholder(tf.int32, [batch_size, None])  # decoder inputs
y_d = tf.placeholder(tf.int32, [batch_size, None])  # reversed loglines [batch_size, num_steps]

visualization_embeddings = tf.Variable(np.zeros([num_examples_to_visualize, state_size]), trainable=False,
                                       name="VisualizationEmbeddings", dtype=DTYPE)

word_embeddings = tf.get_variable('word_embeddings', [vocabulary_size, state_size],
                                  dtype=DTYPE)  # each row is a dense vector for each word.

# Encoder Inputs
encoder_inputs = tf.nn.embedding_lookup(word_embeddings, x_e)  # [batch_size, max_time, embedding_size]
encoder_sequence_lengths = tf.placeholder(tf.int32, [batch_size])  # [batch_size]

# Decoder Inputs
decoder_inputs = tf.nn.embedding_lookup(word_embeddings,
                                        x_d)  # need to be defined  [batch_size, max_time, embedding_size]
decoder_sequence_lengths = tf.placeholder(tf.int32, [batch_size])  # [batch_size]

# Decoder Labels
decoder_labels = y_d  # this are our target words

"""## LSTM, Dropout Wrapper, MultiRNNCell

* https://github.com/tensorflow/tensorflow/blob/v1.0.0-rc0/tensorflow/contrib/rnn/python/ops/core_rnn_cell.py
* https://github.com/tensorflow/tensorflow/blob/v1.0.0-rc0/tensorflow/contrib/rnn/python/ops/lstm_ops.py
* https://github.com/tensorflow/tensorflow/blob/v1.0.0-rc0/tensorflow/contrib/rnn/python/ops/core_rnn_cell_impl.py

* prepare attention states - concatenate output states.
* this is the "old" way of doing things, mayb
* top_states = [array_ops.reshape(e, [-1, 1, encoder_cell.output_size]) for e in encoder_outputs]
* attention_states = array_ops.concat(1, top_states)
"""

from library.core_rnn_cell_impl import \
    DropoutWrapper as DtypeDropoutWrapper  # import v1.1.0 dropout wrapper to support setting DTYPE to half-precision

# Define cells
with tf.variable_scope("encoder_scope") as encoder_scope:
    cell_fw = contrib_rnn.LSTMCell(num_units=state_size, state_is_tuple=True)
    cell_fw = DtypeDropoutWrapper(cell=cell_fw, output_keep_prob=tf_keep_probabiltiy, dtype=DTYPE)
    cell_fw = contrib_rnn.MultiRNNCell(cells=[cell_fw] * num_lstm_layers, state_is_tuple=True)

    cell_bw = contrib_rnn.LSTMCell(num_units=state_size, state_is_tuple=True)
    cell_bw = DtypeDropoutWrapper(cell=cell_bw, output_keep_prob=tf_keep_probabiltiy, dtype=DTYPE)
    cell_bw = contrib_rnn.MultiRNNCell(cells=[cell_bw] * num_lstm_layers, state_is_tuple=True)

    encoder_cell_fw = cell_fw
    encoder_cell_bw = cell_bw

    encoder_outputs, (output_state_fw, output_state_bw) = tf.nn.bidirectional_dynamic_rnn(
        cell_fw=encoder_cell_fw,
        cell_bw=encoder_cell_bw,
        dtype=DTYPE,
        sequence_length=encoder_sequence_lengths,
        inputs=encoder_inputs,
    )

    last_encoder_state = tuple(map(

        lambda fw_state, bw_state: tf.contrib.rnn.LSTMStateTuple(

            c=tf.concat((fw_state.c, bw_state.c), 1,
                        name="bidirectional_concat_c"),

            h=tf.concat((fw_state.h, bw_state.h), 1,
                        name="bidirectional_concat_h")),

        output_state_fw, output_state_bw))

"""## 2.5 Dynamic RNN decoder

* https://github.com/tensorflow/tensorflow/blob/v0.12.0/tensorflow/python/ops/seq2seq.py
* decoder https://github.com/tensorflow/tensorflow/blob/master/tensorflow/contrib/seq2seq/python/ops/seq2seq.py
* legacy attention decoder https://github.com/tensorflow/tensorflow/blob/master/tensorflow/contrib/legacy_seq2seq/python/ops/seq2seq.py
* https://github.com/tensorflow/tensorflow/blob/v1.0.1/tensorflow/python/ops/rnn.py
* https://github.com/tensorflow/tensorflow/blob/v1.0.1/tensorflow/contrib/seq2seq/python/ops/seq2seq.py
* OuputProjectionWrapper https://github.com/tensorflow/tensorflow/blob/v1.0.1/tensorflow/contrib/rnn/python/ops/core_rnn_cell_impl.py#L389
"""

# decoder_outputs = [batch_size, max_time, cell.output_size]
# final_state = [batch_size, cell.state_size]
# final_context_state
with tf.variable_scope("decoder_scope") as decoder_scope:
    # output projection
    # we need to specify output projection manually, because sampled softmax needs to have access to the the projection matrix
    output_projection_w_t = tf.get_variable("output_projection_w", [vocabulary_size, state_size*2], dtype=DTYPE)
    output_projection_w = tf.transpose(output_projection_w_t)
    output_projection_b = tf.get_variable("output_projection_b", [vocabulary_size], dtype=DTYPE)

    # define decoder cell
    decoder_cell = tf.contrib.rnn.LSTMCell(num_units=state_size*2)
    decoder_cell = DtypeDropoutWrapper(cell=decoder_cell, output_keep_prob=tf_keep_probabiltiy, dtype=DTYPE)
    decoder_cell = contrib_rnn.MultiRNNCell(cells=[decoder_cell] * num_lstm_layers, state_is_tuple=True)
    # decoder_cell = contrib_rnn.OutputProjectionWrapper(decoder_cell, output_size=vocabulary_size ) # rnn output: [batch_size, max_time, vocabulary_size ]

    # define decoder train netowrk
    decoder_outputs_tr, _, _ = dynamic_rnn_decoder(  # decoder outputs, final hidden state, final context state
        cell=decoder_cell,  # the cell function
        decoder_fn=simple_decoder_fn_train(last_encoder_state, name=None),
        inputs=decoder_inputs,  # [batch_size, max_time, embedding_size].
        sequence_length=decoder_sequence_lengths,  # length for sequence in the batch [batch_size]
        parallel_iterations=None,  # Tradeoff - time for memory
        swap_memory=False,
        time_major=False)

    # define decoder inference network
    decoder_scope.reuse_variables()

"""# 3 Objective and Optimization

* https://github.com/tensorflow/tensorflow/blob/v1.0.0-rc0/tensorflow/python/ops/nn_impl.py

* The output projection maps from the dimension of the output vector of the LSTM to the vocabualry size.
* The labels come in as matrix [batch_size, time_steps].
* The labels are integers, and denote the row_id of a word in the word embedding.
* The labels are transformed to the shape: [batchsize*timesteps, 1] to perform the sampled softmax.
* The sampled_softmax_function performs the one-hot encoding of the labels.

* Logits are just the outputs of a layer (i.e., sigmoid, tanh...) WITHOUT being squashed by a softmax function
* Explanation of logits.  https://stackoverflow.com/questions/36078411/tensorflow-are-my-logits-in-the-right-format-for-cross-entropy-function

* sparse_softmax_cross_entropy_with_logits  https://github.com/tensorflow/tensorflow/blob/v1.0.1/tensorflow/python/ops/nn_ops.py#L1637

*  https://github.com/tensorflow/tensorflow/blob/v1.0.1/tensorflow/contrib/seq2seq/python/ops/loss.py
* https://www.tensorflow.org/extras/candidate_sampling.pdf

## 3.1 Loss function
"""

print("Decoder Output  Shape: [batch_size=%s, max_timesteps=%s, vocabulary_size=%s]" % tuple(decoder_outputs_tr.shape))
# from library.nn_impl import sampled_softmax_loss as dtype_sampled_softmax_loss # imported cutom loss function to support dtype  as a parameter => half precision

# reshape outputs of decoders to [ batch_size * max_time , vocabulary_size ]
decoder_forward_outputs = tf.reshape(decoder_outputs_tr, [-1, state_size*2])
decoder_target_labels = tf.reshape(decoder_labels, [-1, 1])  # =>  [ batch_size * max_time ]  sequence of correc tlabel

sampled_softmax_losses = tf.nn.sampled_softmax_loss(
    weights=output_projection_w_t,  # [num_classes, state_size]
    biases=output_projection_b,  # [num_classes]
    inputs=decoder_forward_outputs,
    # inputs: A `Tensor` of shape `[batch_size, state_size]`.  The forward activations of the input network.
    labels=decoder_target_labels,
    num_sampled=num_samples,
    num_classes=vocabulary_size,
    num_true=1,
)
total_loss_op = tf.reduce_mean(sampled_softmax_losses)

"""## Regularization"""

l1_regularizer = tf.contrib.layers.l1_regularizer(
    scale=l1_scale, scope=None
)
weights = tf.trainable_variables()
regularization_penalty = tf.contrib.layers.apply_regularization(l1_regularizer, [word_embeddings])

regularized_loss = total_loss_op + regularization_penalty

"""## Trainings step"""

# get gradients for all trainable parameters with respect to our loss funciton
params = tf.trainable_variables()
# optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate)
optimizer = tf.train.RMSPropOptimizer(learning_rate=learning_rate)
gradients = tf.gradients(regularized_loss, params)

# apply gradient clip
clipped_gradients, gradient_norm = tf.clip_by_global_norm(gradients, max_gradient_norm)

# Update operation
training_step = optimizer.apply_gradients(zip(clipped_gradients, params),
                                          global_step=tf_global_step)  # learing rate decay is calulated based on this step

"""# 4 Training

## Prepare Batch Loading for trainings data
"""

# load single example from processed file
filename_queue = tf.train.string_input_producer([processed_datafile], num_epochs=None)  # num_epochs=None
reader = tf.TextLineReader()
example_id, single_example_logline = reader.read(filename_queue)

# define how to parse single example (see 1.3)
split_example = tf.string_split([single_example_logline], delimiter="|")
split_example_dense = tf.sparse_tensor_to_dense(split_example, default_value='', validate_indices=True, name=None)
split_example_dense = split_example_dense[0]

# split_example_dense[0] is signature id
# sequence length
sequence_length = tf.string_to_number(split_example_dense[1], out_type=tf.int32)
# split encoder inputs
enc_split = tf.string_split([split_example_dense[2]], delimiter=",")
enc_split_dense = tf.sparse_tensor_to_dense(enc_split, default_value='', validate_indices=True, name=None)
x_e_single = tf.string_to_number(enc_split_dense, out_type=tf.int32)
# split decoder inputs
dec_split = tf.string_split([split_example_dense[3]], delimiter=",")
dec_split_dense = tf.sparse_tensor_to_dense(dec_split, default_value='', validate_indices=True, name=None)
x_d_single = tf.string_to_number(dec_split_dense, out_type=tf.int32)
# split decoder targets
tar_split = tf.string_split([split_example_dense[4]], delimiter=",")
tar_split_dense = tf.sparse_tensor_to_dense(tar_split, default_value='', validate_indices=True, name=None)
y_single = tf.string_to_number(tar_split_dense, out_type=tf.int32)

# Batch the variable length tensor with dynamic padding
fetch_trainings_batch = tf.train.batch(
    tensors=[
        x_e_single[0],  # single encoder input line
        x_d_single[0],  # single decoder input line
        y_single[0],  # single decoder target line
        sequence_length,  # sequence length of this example
    ],
    batch_size=batch_size,
    dynamic_pad=True,
    name="trainings_batch",
    allow_smaller_final_batch=True
)

"""# 4.1 Session Setup

* https://github.com/tensorflow/tensorflow/blob/master/tensorflow/python/ops/data_flow_ops.py
* https://github.com/tensorflow/tensorflow/blob/master/tensorflow/g3doc/api_docs/python/functions_and_classes/shard8/tf.train.batch.md
* https://www.tensorflow.org/how_tos/summaries_and_tensorboard/
"""


def get_batch_dict(session):
    # get trainingsbatch from input queue
    batch = session.run([fetch_trainings_batch], feed_dict=None)
    batch = batch[0]

    # assign arrays to dictionary
    batch_dict = {
        x_e: batch[0],  # encoder inputs
        x_d: batch[1],  # decoder inputs
        y_d: batch[2],  # decoder targets
        encoder_sequence_lengths: batch[3],
        decoder_sequence_lengths: batch[3]
    }
    return batch_dict


"""# 4.2  Trainings Loop"""

if learn_model:
    current_epoch = 0
    save_checkpoint_after_each_step = int(max_steps / 10)
    print_loss_after_steps = int(max_steps / 100)

    queue_capacity = 2 * batch_size

    # Saver
    saver = tf.train.Saver(tf.global_variables())

    # Summaries
    tf.summary.scalar("total_loss", tf.cast(total_loss_op, DTYPE))  # summary for accuracy
    tf.summary.scalar("regularized_loss", tf.cast(regularized_loss, DTYPE))  # summary for accuracy
    # Start session
    session = tf.Session()

    # Check random seed for reproducability
    control_random_number = session.run(tf.random_normal([1]))
    print("Control Random Number: %0.5f" % control_random_number)  # should change if you modify the graph

    all_summaries = tf.summary.merge_all()
    # summary_writer = tf.summary.FileWriter(graph_dir, graph=session.graph)

    session.run([
        tf.local_variables_initializer(),
        tf.global_variables_initializer(),
    ])

s = time.time()  # start time
if learn_model:

    word_embeddings_before = session.run(word_embeddings)
    import collections

    batch_times = collections.deque()

    coord = tf.train.Coordinator()
    threads = tf.train.start_queue_runners(sess=session, coord=coord)

    for current_step in range(1, max_steps + 1):  # start from 1 .. max_steps+1 to execute max steps

        try:
            step_s = time.time()
            # increase step counter
            session.run(tf_global_step.assign(current_step))

            # get next batch
            batch_dict = get_batch_dict(session)
            # execute actions
            results = session.run([
                total_loss_op,  # calculate training loss
                regularized_loss,  #
                training_step,  # calculate gradients, update gradients
                all_summaries,  # compile summaries and write to graph dir,
                learning_rate,
                # get gradients
            ], feed_dict=batch_dict)

            # summary_writer.add_summary(results[3], current_step)
            batch_times.append((time.time() - step_s))

        except Exception as e:
            # Report exceptions to the coordinator.
            print("Aborting trainingsloop :%s" % str(e.message))
            coord.request_stop(e)
            break

        if current_step % print_loss_after_steps == 0:
            print(
                "Epoch {epoch:02d}, Step {current_step:05d}/{max_steps:05d}, Current Learning rate: {learning_rate:0.4f},  Loss: {loss:0.4f} Regularized Loss: {regularized_loss:0.4f}".format(
                    epoch=int(current_step / (max_steps / epochs) + 1),
                    current_step=current_step, max_steps=max_steps,
                    learning_rate=results[4],
                    loss=results[0], regularized_loss=results[1]
                )  # end format
            )  # end print
            avg_batch_time = sum(batch_times) / len(batch_times)
            total_time_in_s = avg_batch_time * max_steps
            print("Average step time: %0.2fs, Estimated total duration: ~%0.2f min (~ %0.2f h) " % (
                avg_batch_time, total_time_in_s / 60.0, total_time_in_s / 3600.0))

        # save checkpoint every xth step
        # if current_step % save_checkpoint_after_each_step == 0:
        #     print("Saving checkpoint")
        #
        #     chkpoint_out_filename = join_path(checkpoint_path, model_name)
        #     saver.save(session, chkpoint_out_filename, global_step=current_step)

            # stop training queues
    coord.request_stop()
    word_embeddings_after = session.run(word_embeddings)
    e = time.time()
    print("Learning took {sec:0.2f} seconds".format(sec=(e - s)))
else:
    pass
    # current_step = 35821  # TODO this needs to be adjusted
    #
    # session = tf.Session()
    # meta_data = join_path(result_dir, checkpoint_path, 'rnn-autoencoder-35821.meta')  # TODO this needs to be adjusted
    # print(meta_data)
    # saver = tf.train.Saver()  # tf.train.import_meta_graph(meta_data)
    # print("Restored graph")
    # model_to_load = tf.train.latest_checkpoint(join_path(result_dir, checkpoint_path))
    # print("Load '%s' from saved checkpoints" % model_to_load)
    # saver.restore(session, model_to_load)

"""# 5 Visualization

* https://colah.github.io/posts/2015-01-Visualizing-Representations/
* https://github.com/normanheckscher/mnist-tensorboard-embeddings/blob/master/mnist_t-sne.py
* https://www.tensorflow.org/how_tos/embedding_viz/

**Embedd test loglines 2000**
"""

embedded_csv = join_path(result_dir, experiment_dir, "embedded_lines.csv")
embedded_loglines = np.ndarray(shape=[num_examples_to_embed, state_size*2])

if not os.path.exists(embedded_csv):
    # load
    test_examples = list(open(test_data_processed, "r"))  # ------------------changed to test data-----------------
    print(len(test_examples))
    test_examples = [t.split("|") for t in test_examples]  # ------------------changed to test data-----------------
    print("Loaded trainingsdata")

    s = time.time()
    count = 0
    train_batches = list(range(0, num_examples_to_embed, batch_size))
    print("Started embedding %i batches of %i log lines" % (len(train_batches), num_examples_to_embed))
    for i, ex_id in enumerate(train_batches):
        print("Embedded %i ercent of batches" % (i))
        # if (i+1)%(int(len(train_batches)/20))==0:  ---------------commented since not essencial-----------------------
        # print ("Embedded %0.2f ercent of batches"%(100*i/float(len(train_batches)))) ---------------commented since not essencial-----------------------
        x_e_batch = []
        esl_batch = []

        incomplete_batch = False  # for last batch
        incomplete_length = 0
        # get batch, write metadata
        for b_id, example in enumerate(test_examples[
                                       ex_id:ex_id + batch_size]):  # --------------------- training_examples => test_examples---------------
            count += 1
            esl_batch.append(int(example[1]))  # sequence length
            x_e_batch.append(np.array(example[2].split(","), dtype=np.integer))  # encoder input

        if len(x_e_batch) < batch_size:  # last_batch, so need to zero add id
            incomplete_batch = True
            incomplete_length = len(x_e_batch)

            for _ in range(batch_size - len(x_e_batch)):
                esl_batch.append(0)
                x_e_batch.append(np.zeros([len(x_e_batch[0])]))

        # pad batch
        max_seq_len = np.amax(esl_batch)  # maximum encoder length
        padded_x_e_batch = []
        for x_es in x_e_batch:
            padded_x_e = np.pad(x_es, (0, max_seq_len - len(x_es)), 'constant', constant_values=0)
            padded_x_e_batch.append(padded_x_e)

            # assign arrays to dictionary
        batch_dict = {
            x_e: np.array(padded_x_e_batch),  # encoder inputs  # [batch_size, max_sequence_length, state_size ]
            encoder_sequence_lengths: np.array(esl_batch)  # encoder sequence length [batch_size,1]
        }

        # execute actions
        results = session.run([
            last_encoder_state,  # get encoded trainings samples
        ], feed_dict=batch_dict)

        # story in temporary array
        if incomplete_batch:
            embedded_loglines[ex_id:ex_id + incomplete_length, ] = results[0][0].c[0:incomplete_length]
        else:
            embedded_loglines[ex_id:ex_id + batch_size, ] = results[0][0].c  # copy c hidden state to tmp_ary

    print("Embedded %i lines in ~%0.2f min" % (num_examples_to_embed, ((time.time() - s) / 60.0)))
    # fp = np.memmap(embedded_csv, dtype='float32', mode='w+', shape=(num_examples_to_embed,state_size))
    # fp[:]=embedded_loglines[:]
    # fp.flush()
else:
    # fp = np.memmap(embedded_csv, dtype='float32', mode='r', shape=(num_examples_to_embed,state_size))
    embedded_loglines = np.genfromtxt(embedded_csv, delimiter=";")
    print("Loaded embedded lines to file: %s " % embedded_csv)
    print(embedded_loglines.shape)

    # embedded_loglines[:]=fp[:]
X = embedded_loglines
print(X)

np.savetxt(result_dir + '/' + experiment_dir + "/" + dataset + "_" +experiment_sub_dir + "_" + "embeddings.csv", X, delimiter=",")
# print(result_dir + "/" + experiment_dir + "/" + dataset + "_" +experiment_sub_dir + "_embeddings.csv")
# if os.path.exists(result_dir + "/" + experiment_dir + "/" + dataset + "_" +experiment_sub_dir + "_embeddings.csv"):
#     X = pd.read_csv(result_dir + "/" + experiment_dir + "/" + dataset + "_" +experiment_sub_dir + "_embeddings.csv", header=None).to_numpy()


results/biLSTM_encoder_2 directory was created
Running experiment: apache
Loaded 56482 loglines
Tokenizing 56482 lines... 
Estimated time for tokenization ~ 0.01 (in min)
Tokenized logfile.
Created index_to_word and word_to_index dictionary. 
Saved to json: results/biLSTM_encoder_2/18_apache/vocabulary.json.
Saved to json: results/biLSTM_encoder_2/18_apache/tokenized_loglines.json.
Saved to json: results/biLSTM_encoder_2/18_apache/index_to_word.json.
Saved to json: results/biLSTM_encoder_2/18_apache/word_to_index.json.
Tokenized loglines samples:
	['LOG_START', '', '[', 'thu', 'jun', '09', '06', ':', '07', ':', '04', '2005', ']', '', '', '[', 'notice', ']', '', 'ldap', ':', '', 'built', 'with', 'openldap', 'ldap', 'sdk\n', 'LOG_END']

	['LOG_START', '', '[', 'thu', 'jun', '09', '06', ':', '07', ':', '04', '2005', ']', '', '', '[', 'notice', ']', '', 'ldap', ':', '', 'ssl', 'support', 'unavailable\n', 'LOG_END']

	['LOG_START', '', '[', 'thu', 'jun', '09', '06', ':', '07', ':', '04', '2

Epoch 01, Step 00135/00564, Current Learning rate: 0.0180,  Loss: 5.7008 Regularized Loss: 5.7008
Average step time: 7.16s, Estimated total duration: ~67.30 min (~ 1.12 h) 
Epoch 01, Step 00140/00564, Current Learning rate: 0.0180,  Loss: 4.8694 Regularized Loss: 4.8694
Average step time: 7.14s, Estimated total duration: ~67.10 min (~ 1.12 h) 
Epoch 01, Step 00145/00564, Current Learning rate: 0.0180,  Loss: 4.7556 Regularized Loss: 4.7556
Average step time: 7.10s, Estimated total duration: ~66.76 min (~ 1.11 h) 
Epoch 01, Step 00150/00564, Current Learning rate: 0.0180,  Loss: 3.8961 Regularized Loss: 3.8961
Average step time: 7.10s, Estimated total duration: ~66.71 min (~ 1.11 h) 
Epoch 01, Step 00155/00564, Current Learning rate: 0.0180,  Loss: 4.2227 Regularized Loss: 4.2227
Average step time: 7.13s, Estimated total duration: ~67.05 min (~ 1.12 h) 
Epoch 01, Step 00160/00564, Current Learning rate: 0.0180,  Loss: 4.1872 Regularized Loss: 4.1872
Average step time: 7.14s, Estimated t

Epoch 02, Step 00375/00564, Current Learning rate: 0.0147,  Loss: 1.1438 Regularized Loss: 1.1438
Average step time: 7.22s, Estimated total duration: ~67.86 min (~ 1.13 h) 
Epoch 02, Step 00380/00564, Current Learning rate: 0.0147,  Loss: 1.2812 Regularized Loss: 1.2812
Average step time: 7.20s, Estimated total duration: ~67.71 min (~ 1.13 h) 
Epoch 02, Step 00385/00564, Current Learning rate: 0.0147,  Loss: 1.2124 Regularized Loss: 1.2124
Average step time: 7.19s, Estimated total duration: ~67.61 min (~ 1.13 h) 
Epoch 02, Step 00390/00564, Current Learning rate: 0.0147,  Loss: 1.1881 Regularized Loss: 1.1881
Average step time: 7.19s, Estimated total duration: ~67.61 min (~ 1.13 h) 
Epoch 02, Step 00395/00564, Current Learning rate: 0.0140,  Loss: 1.4344 Regularized Loss: 1.4344
Average step time: 7.18s, Estimated total duration: ~67.53 min (~ 1.13 h) 
Epoch 02, Step 00400/00564, Current Learning rate: 0.0140,  Loss: 1.4610 Regularized Loss: 1.4610
Average step time: 7.18s, Estimated t

In [41]:

"""**Homogenity, Completeness, V-Measure**"""

cluster_alg = "birch"

true_labels_file_path = "data/" + dataset + ".ids"
embeddings_file_path = result_dir + '/' + experiment_dir + "/" + dataset + "_" +experiment_sub_dir + "_" + "embeddings.csv"



print(embeddings_file_path)
print(labels_true_file)









results/biLSTM_encoder_2/18_apache/apache_biLSTM_encoder_2_embeddings.csv
data/apache.ids


In [48]:
file = open(true_labels_file_path, "r")
true_labels_str_arr = file.read().splitlines()
true_labels_str_arr

['2',
 '3',
 '1',
 '1',
 '1',
 '2',
 '2',
 '2',
 '3',
 '3',
 '3',
 '1',
 '1',
 '1',
 '2',
 '2',
 '3',
 '1',
 '1',
 '1',
 '1',
 '2',
 '2',
 '2',
 '3',
 '3',
 '3',
 '1',
 '1',
 '1',
 '2',
 '2',
 '3',
 '3',
 '1',
 '1',
 '1',
 '2',
 '2',
 '3',
 '3',
 '1',
 '1',
 '1',
 '2',
 '3',
 '1',
 '1',
 '1',
 '1',
 '2',
 '2',
 '2',
 '2',
 '3',
 '3',
 '3',
 '3',
 '1',
 '2',
 '3',
 '1',
 '1',
 '2',
 '2',
 '3',
 '3',
 '1',
 '1',
 '2',
 '2',
 '3',
 '3',
 '1',
 '2',
 '3',
 '1',
 '2',
 '3',
 '1',
 '1',
 '2',
 '2',
 '3',
 '3',
 '1',
 '2',
 '3',
 '1',
 '1',
 '2',
 '3',
 '2',
 '3',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '1',
 '2',
 '3',
 '2',
 '3',
 '2',
 '3',
 '2',
 '3',
 '2',
 '3',
 '2',
 '3',
 '2',
 '3',
 '1',
 '2',
 '3',
 '1',
 '1',
 '2',
 '3',
 '2',
 '3',
 '1',
 '2',
 '3',
 '4',
 '1',
 '2',
 '3',
 '1',
 '1',
 '2',
 '2',
 '3',
 '1',
 '1',
 '1',
 '2',
 '3',
 '1',
 '1',
 '1',
 '1',
 '2',
 '3',
 '2',
 '3',
 '2',
 '3',
 '2',
 '3',
 '1',
 '1',
 '1',
 '2',
 '2',
 '3',
 '3',
 '1',
 '1',
 '1'

In [50]:

clusters = np.array([int(i) for i in true_labels_str_arr])
clusters



array([2, 3, 1, ..., 1, 2, 3])

In [52]:
embeddingds_df = pd.read_csv(embeddings_file_path, header=None)
X = embeddingds_df.values
X

array([[ 1.38971305e+00,  7.02811193e+00,  8.84187639e-01, ...,
        -5.35470899e-03,  3.06327868e+00, -6.14787483e+00],
       [-2.11520648e+00,  3.94894195e+00, -2.82020748e-01, ...,
        -4.82805446e-03,  3.11748099e+00, -5.65914011e+00],
       [-1.64294159e+00,  4.91066504e+00, -1.49028614e-01, ...,
        -5.30686043e-03,  3.10822821e+00, -5.89561319e+00],
       ...,
       [-1.64240909e+00,  4.64230299e+00, -1.50913566e-01, ...,
        -5.32858726e-03,  3.11910057e+00, -5.87741423e+00],
       [ 1.39055681e+00,  7.06767178e+00,  8.84063065e-01, ...,
        -5.36920782e-03,  3.06923676e+00, -6.14552736e+00],
       [-2.08254099e+00,  3.90954065e+00, -1.72771350e-01, ...,
        -5.04545867e-03,  3.13206339e+00, -5.66250610e+00]])

In [53]:
min_thres = 0.1
max_thres = 30
thres_step = 0.2

predicted_labels = {}
cluster_results = {}

for i in np.arange(min_thres, max_thres, thres_step):
    row = []
    if (cluster_alg == "birch"):
        brc = Birch(threshold=i, branching_factor=50, n_clusters=None, compute_labels=True, copy=True)
        brc.fit(X)
        labels = brc.predict(X)
    elif cluster_alg == "dbscan":
        print("dbscan")
        break

    hom_com_vmes = homogeneity_completeness_v_measure(clusters, labels)

    shilloutte = silhouette_score(X, clusters)
    mutualinfoscore = mutual_info_score(clusters, labels, contingency=None)
    adjustedmutualinfoscore = adjusted_mutual_info_score(clusters, labels)
    # daviesbouldinscore = davies_bouldin_score(X, clusters)

    # print("adjustedmutualinfoscore",adjustedmutualinfoscore)
    # f1_score = f1Score(clusters,labels)

    row += list(hom_com_vmes)
    row.append(shilloutte)
    # row.append(f1_score)
    row.append(mutualinfoscore)
    row.append(adjustedmutualinfoscore)
    # row.append(daviesbouldinscore)
    try:
        predicted_labels[i].append(labels)
    except KeyError:
        predicted_labels[i] = labels
    cluster_results[i] = row
print(predicted_labels)
print(cluster_results)

print(len(cluster_results))


{0.1: array([1236,  835,  304, ...,  303, 1212, 1058]), 0.30000000000000004: array([ 68, 698, 359, ..., 413,  84, 949]), 0.5000000000000001: array([631,  79, 363, ..., 422, 625, 185]), 0.7000000000000001: array([408,  52, 251, ..., 337, 408, 157]), 0.9000000000000001: array([236, 183,  50, ...,  94, 236, 210]), 1.1000000000000003: array([142,  10,  94, ..., 104, 142,   8]), 1.3000000000000003: array([85,  0, 38, ..., 54, 85,  1]), 1.5000000000000004: array([0, 2, 3, ..., 3, 0, 6]), 1.7000000000000004: array([0, 1, 2, ..., 7, 0, 4]), 1.9000000000000004: array([0, 1, 2, ..., 2, 0, 4]), 2.1000000000000005: array([0, 1, 2, ..., 2, 0, 4]), 2.3000000000000007: array([0, 1, 2, ..., 2, 0, 5]), 2.5000000000000004: array([0, 1, 2, ..., 2, 0, 5]), 2.7000000000000006: array([0, 1, 2, ..., 2, 0, 4]), 2.900000000000001: array([0, 1, 2, ..., 2, 0, 1]), 3.1000000000000005: array([0, 1, 2, ..., 2, 0, 3]), 3.3000000000000007: array([0, 1, 2, ..., 2, 0, 1]), 3.500000000000001: array([0, 1, 2, ..., 2, 0, 

In [54]:
for p_l in predicted_labels:
    print(p_l, len(np.unique(predicted_labels[p_l])), cluster_results[p_l][2])




0.1 1387 0.2904015891859272
0.30000000000000004 987 0.3073577400196835
0.5000000000000001 664 0.33923154351292306
0.7000000000000001 427 0.37590004575258096
0.9000000000000001 246 0.413320726156999
1.1000000000000003 149 0.4532772936269883
1.3000000000000003 87 0.5215558708438954
1.5000000000000004 52 0.5776926644587048
1.7000000000000004 35 0.6354599868825869
1.9000000000000004 23 0.6739909041817389
2.1000000000000005 20 0.6938037155093384
2.3000000000000007 14 0.7471265971056334
2.5000000000000004 14 0.759881073098913
2.7000000000000006 9 0.8281263577693913
2.900000000000001 14 0.7847237828623447
3.1000000000000005 9 0.8141354872083532
3.3000000000000007 6 1.0000000000000002
3.500000000000001 6 1.0000000000000002
3.7000000000000006 5 0.9867335412617743
3.900000000000001 5 0.9867335412617743
4.1000000000000005 5 0.9867335412617743
4.300000000000001 5 0.9867335412617743
4.500000000000001 5 0.9867335412617743
4.7 5 0.9867335412617743
4.9 5 0.9867335412617743
5.1000000000000005 5 0.98673

In [55]:
import matplotlib.pyplot as plt

homogenity = []
completeness = []
vScore = []
shilloutte = []
f_score = []
mutualinfoscore = []
adjustedmutualinfoscore = []

y = list(cluster_results.keys())
print("Threshold : ", list(y))



Threshold :  [0.1, 0.30000000000000004, 0.5000000000000001, 0.7000000000000001, 0.9000000000000001, 1.1000000000000003, 1.3000000000000003, 1.5000000000000004, 1.7000000000000004, 1.9000000000000004, 2.1000000000000005, 2.3000000000000007, 2.5000000000000004, 2.7000000000000006, 2.900000000000001, 3.1000000000000005, 3.3000000000000007, 3.500000000000001, 3.7000000000000006, 3.900000000000001, 4.1000000000000005, 4.300000000000001, 4.500000000000001, 4.7, 4.9, 5.1000000000000005, 5.300000000000001, 5.500000000000001, 5.700000000000001, 5.9, 6.1000000000000005, 6.300000000000001, 6.500000000000001, 6.700000000000001, 6.900000000000001, 7.100000000000001, 7.300000000000001, 7.500000000000001, 7.700000000000001, 7.900000000000001, 8.100000000000001, 8.3, 8.500000000000002, 8.700000000000001, 8.900000000000002, 9.100000000000001, 9.3, 9.500000000000002, 9.700000000000001, 9.900000000000002, 10.100000000000001, 10.300000000000002, 10.500000000000002, 10.700000000000001, 10.900000000000002, 

In [56]:
for val in cluster_results.items():
    homogenity += [val[1][0]]
    completeness += [val[1][1]]
    vScore += [val[1][2]]
    shilloutte += [val[1][3]]
    # f_score+=[val[1][4]]
    mutualinfoscore += [val[1][4]]
    adjustedmutualinfoscore += [val[1][5]]

# print("F1 Score : ", f_score)
print("Homogenity : ", homogenity)
print("Completeness : ", completeness)
print("V-Score : ", vScore)
print("Shilloutte", shilloutte)
print("Mutual info score", mutualinfoscore)
print("Adjusted mutual info score", adjustedmutualinfoscore)




Homogenity :  [1.0000000000000004, 1.0000000000000004, 1.0000000000000002, 1.0000000000000004, 1.0000000000000004, 1.0000000000000004, 1.0000000000000002, 1.0000000000000004, 1.0000000000000002, 1.0000000000000004, 1.0, 1.0000000000000002, 1.0000000000000004, 1.0000000000000004, 1.0000000000000002, 1.0000000000000002, 1.0000000000000002, 1.0000000000000002, 0.9738144717535681, 0.9738144717535681, 0.9738144717535681, 0.9738144717535681, 0.9738144717535681, 0.9738144717535681, 0.9738144717535681, 0.9738144717535681, 0.9738144717535681, 0.9738144717535681, 0.9497982738937256, 0.9497982738937256, 0.9497982738937256, 0.9497982738937256, 0.9497982738937256, 0.9497982738937256, 0.9497982738937256, 0.9497982738937256, 0.9497982738937256, 0.9497982738937256, 0.9497982738937256, 0.9497982738937256, 0.9497982738937256, 0.9497982738937256, 0.9497982738937256, 0.9497982738937256, 0.9497982738937256, 0.9497982738937256, 0.9497982738937256, 0.9497982738937256, 0.9497982738937256, 0.9497982738937256, 

In [59]:

# plt.plot(y,f_score, label = "F1 Score")
plt.plot(y, homogenity, label="Homogenity")
plt.plot(y, completeness, label="Completeness")
plt.plot(y, vScore, label="V-Score")
plt.plot(y, shilloutte, label="Shilloutte")
plt.plot(y, mutualinfoscore, label="Mutual_info_score")
plt.plot(y, adjustedmutualinfoscore, label="Adjusted mutual info score")

plt.xlabel('Threshold')

plt.legend()

plt.show()




In [58]:
print("Highest V-cscore :", y[vScore.index(max(vScore))])

Highest V-cscore : 3.3000000000000007


In [60]:

"""**Write Clustering Results**"""

clustering_Results = {
    'Threshold': y,
    'Homogenity': homogenity,
    'Completeness': completeness,
    'vScore': vScore,
    'Shilloutte': shilloutte,
    'Mutual info score': mutualinfoscore,
    'Adjusted mutual info score': adjustedmutualinfoscore
}

results = pd.DataFrame(clustering_Results)

results.to_csv(join_path(result_dir, experiment_dir, dataset + '_clustering_results.csv'))


In [61]:
from itertools import groupby

def getTemplate(candidate):
    # candidate: list of list
    abstraction = ''

    # transpose row to column
    candidate_transpose = list(zip(*candidate))
    candidate_length = len(candidate)

    if candidate_length > 1:
        # get abstraction
        abstraction_list = []
        for index, message in enumerate(candidate_transpose):
            message_length = len(set(message))
            if message_length == 1:
                if ('/' in message[0]):
                    abstraction_list.append('<*>')
                elif ('-' in message[0]):
                    if (message[0][0] == '-'):
                        abstraction_list[-1] = '<*>'
                    abstraction_list.append('<*>')
                else:
                    abstraction_list.append(message[0])
            else:
                if (message[0][0] == '['):
                    abstraction_list.append('[<*>')
                elif (message[0][-1] == ']'):
                    abstraction_list.append('<*>]')
                else:
                    abstraction_list.append('<*>')

        abstraction = ' '.join(abstraction_list)

    elif candidate_length == 1:
        abstraction = ' '.join(candidate[0])

    return abstraction


In [62]:
"""**Read Ground ruth templates**"""
import pandas as pd

df_raw_logs = pd.read_csv("gt/" + dataset + "_2k.log_structured.csv")
raw_logs = df_raw_logs['Content'].to_numpy()

raw_logs



array(['workerEnv.init() ok /etc/httpd/conf/workers2.properties',
       'mod_jk child workerEnv in error state 6',
       'jk2_init() Found child 6725 in scoreboard slot 10', ...,
       'jk2_init() Found child 6790 in scoreboard slot 7',
       'workerEnv.init() ok /etc/httpd/conf/workers2.properties',
       'mod_jk child workerEnv in error state 6'], dtype=object)

In [63]:

len(raw_logs)

2000

In [64]:

"""**Get best clusterong based config**"""


predicted_labels = predicted_labels[y[vScore.index(max(vScore))]]
predicted_labels



array([0, 1, 2, ..., 2, 0, 1])

In [ ]:

# groups = pd.DataFrame()
# groups['predicted_labels'] = predicted_labels
# groups['raw_logs'] = raw_logs
# groups


In [16]:
# for i in np.unique(groups.predicted_labels):
#     print(i)
#     for index, row in groups.iterrows():
#         if row['predicted_labels'] == i:
#             print(i,row['raw_logs'])
#     print('\n')


0
0 workerEnv.init() ok /etc/httpd/conf/workers2.properties
0 workerEnv.init() ok /etc/httpd/conf/workers2.properties
0 workerEnv.init() ok /etc/httpd/conf/workers2.properties
0 workerEnv.init() ok /etc/httpd/conf/workers2.properties
0 workerEnv.init() ok /etc/httpd/conf/workers2.properties
0 workerEnv.init() ok /etc/httpd/conf/workers2.properties
0 workerEnv.init() ok /etc/httpd/conf/workers2.properties
0 workerEnv.init() ok /etc/httpd/conf/workers2.properties
0 workerEnv.init() ok /etc/httpd/conf/workers2.properties
0 workerEnv.init() ok /etc/httpd/conf/workers2.properties
0 workerEnv.init() ok /etc/httpd/conf/workers2.properties
0 workerEnv.init() ok /etc/httpd/conf/workers2.properties
0 workerEnv.init() ok /etc/httpd/conf/workers2.properties
0 workerEnv.init() ok /etc/httpd/conf/workers2.properties
0 workerEnv.init() ok /etc/httpd/conf/workers2.properties
0 workerEnv.init() ok /etc/httpd/conf/workers2.properties
0 workerEnv.init() ok /etc/httpd/conf/workers2.properties
0 workerEnv.

0 workerEnv.init() ok /etc/httpd/conf/workers2.properties
0 workerEnv.init() ok /etc/httpd/conf/workers2.properties
0 workerEnv.init() ok /etc/httpd/conf/workers2.properties
0 workerEnv.init() ok /etc/httpd/conf/workers2.properties
0 workerEnv.init() ok /etc/httpd/conf/workers2.properties
0 workerEnv.init() ok /etc/httpd/conf/workers2.properties
0 workerEnv.init() ok /etc/httpd/conf/workers2.properties
0 workerEnv.init() ok /etc/httpd/conf/workers2.properties
0 workerEnv.init() ok /etc/httpd/conf/workers2.properties
0 workerEnv.init() ok /etc/httpd/conf/workers2.properties
0 workerEnv.init() ok /etc/httpd/conf/workers2.properties
0 workerEnv.init() ok /etc/httpd/conf/workers2.properties
0 workerEnv.init() ok /etc/httpd/conf/workers2.properties
0 workerEnv.init() ok /etc/httpd/conf/workers2.properties
0 workerEnv.init() ok /etc/httpd/conf/workers2.properties
0 workerEnv.init() ok /etc/httpd/conf/workers2.properties
0 workerEnv.init() ok /etc/httpd/conf/workers2.properties
0 workerEnv.in

1 mod_jk child workerEnv in error state 6
1 mod_jk child workerEnv in error state 8
1 mod_jk child workerEnv in error state 9
1 mod_jk child workerEnv in error state 6
1 mod_jk child workerEnv in error state 6
1 mod_jk child workerEnv in error state 6
1 mod_jk child workerEnv in error state 9
1 mod_jk child workerEnv in error state 8
1 mod_jk child workerEnv in error state 9
1 mod_jk child workerEnv in error state 6
1 mod_jk child workerEnv in error state 6
1 mod_jk child workerEnv in error state 6
1 mod_jk child workerEnv in error state 6
1 mod_jk child workerEnv in error state 8
1 mod_jk child workerEnv in error state 7
1 mod_jk child workerEnv in error state 9
1 mod_jk child workerEnv in error state 8
1 mod_jk child workerEnv in error state 8
1 mod_jk child workerEnv in error state 6
1 mod_jk child workerEnv in error state 6
1 mod_jk child workerEnv in error state 9
1 mod_jk child workerEnv in error state 10
1 mod_jk child workerEnv in error state 8
1 mod_jk child workerEnv in error

1 mod_jk child workerEnv in error state 6
1 mod_jk child workerEnv in error state 6
1 mod_jk child workerEnv in error state 7
1 mod_jk child workerEnv in error state 6
1 mod_jk child workerEnv in error state 7
1 mod_jk child workerEnv in error state 6
1 mod_jk child workerEnv in error state 7
1 mod_jk child workerEnv in error state 6
1 mod_jk child workerEnv in error state 7
1 mod_jk child workerEnv in error state 6
1 mod_jk child workerEnv in error state 6
1 mod_jk child workerEnv in error state 6
1 mod_jk child workerEnv in error state 6
1 mod_jk child workerEnv in error state 6
1 mod_jk child workerEnv in error state 6
1 mod_jk child workerEnv in error state 6
1 mod_jk child workerEnv in error state 6
1 mod_jk child workerEnv in error state 6
1 mod_jk child workerEnv in error state 6
1 mod_jk child workerEnv in error state 6
1 mod_jk child workerEnv in error state 6
1 mod_jk child workerEnv in error state 6
1 mod_jk child workerEnv in error state 6
1 mod_jk child workerEnv in error 

2 jk2_init() Found child 1387 in scoreboard slot 7
2 jk2_init() Found child 1390 in scoreboard slot 10
2 jk2_init() Found child 1388 in scoreboard slot 8
2 jk2_init() Found child 1389 in scoreboard slot 9
2 jk2_init() Found child 1393 in scoreboard slot 8
2 jk2_init() Found child 1395 in scoreboard slot 10
2 jk2_init() Found child 1396 in scoreboard slot 6
2 jk2_init() Found child 1394 in scoreboard slot 9
2 jk2_init() Found child 1397 in scoreboard slot 7
2 jk2_init() Found child 1414 in scoreboard slot 7
2 jk2_init() Found child 1412 in scoreboard slot 10
2 jk2_init() Found child 1413 in scoreboard slot 6
2 jk2_init() Found child 1448 in scoreboard slot 6
2 jk2_init() Found child 1439 in scoreboard slot 7
2 jk2_init() Found child 1441 in scoreboard slot 9
2 jk2_init() Found child 1450 in scoreboard slot 11
2 jk2_init() Found child 1449 in scoreboard slot 10
2 jk2_init() Found child 1440 in scoreboard slot 8
2 jk2_init() Found child 1452 in scoreboard slot 7
2 jk2_init() Found child 1

2 jk2_init() Found child 5925 in scoreboard slot 8
2 jk2_init() Found child 5923 in scoreboard slot 6
2 jk2_init() Found child 5924 in scoreboard slot 7
2 jk2_init() Found child 5922 in scoreboard slot 9
2 jk2_init() Found child 5935 in scoreboard slot 9
2 jk2_init() Found child 5936 in scoreboard slot 6
2 jk2_init() Found child 5938 in scoreboard slot 8
2 jk2_init() Found child 5937 in scoreboard slot 7
2 jk2_init() Found child 5940 in scoreboard slot 6
2 jk2_init() Found child 5939 in scoreboard slot 9
2 jk2_init() Found child 5942 in scoreboard slot 8
2 jk2_init() Found child 5941 in scoreboard slot 7
2 jk2_init() Found child 5954 in scoreboard slot 7
2 jk2_init() Found child 5953 in scoreboard slot 6
2 jk2_init() Found child 5956 in scoreboard slot 9
2 jk2_init() Found child 5957 in scoreboard slot 6
2 jk2_init() Found child 5955 in scoreboard slot 8
2 jk2_init() Found child 5961 in scoreboard slot 6
2 jk2_init() Found child 5960 in scoreboard slot 9
2 jk2_init() Found child 5968 i

3 [client 221.232.178.24] Directory index forbidden by rule: /var/www/html/
3 [client 207.12.15.211] Directory index forbidden by rule: /var/www/html/
3 [client 141.153.150.164] Directory index forbidden by rule: /var/www/html/
3 [client 198.232.168.9] Directory index forbidden by rule: /var/www/html/
3 [client 67.166.248.235] Directory index forbidden by rule: /var/www/html/
3 [client 141.154.18.244] Directory index forbidden by rule: /var/www/html/
3 [client 216.216.185.130] Directory index forbidden by rule: /var/www/html/
3 [client 218.75.106.250] Directory index forbidden by rule: /var/www/html/
3 [client 68.228.3.15] Directory index forbidden by rule: /var/www/html/
3 [client 61.220.139.68] Directory index forbidden by rule: /var/www/html/


4
4 mod_jk child init 1 -2
4 mod_jk child init 1 -2
4 mod_jk child init 1 -2
4 mod_jk child init 1 -2
4 mod_jk child init 1 -2
4 mod_jk child init 1 -2
4 mod_jk child init 1 -2
4 mod_jk child init 1 -2
4 mod_jk child init 1 -2
4 mod_jk child 

In [20]:
# groups.head(10)

,predicted_labels,raw_logs
0,0,workerEnv.init() ok /etc/httpd/conf/workers2.p...
1,1,mod_jk child workerEnv in error state 6
2,2,jk2_init() Found child 6725 in scoreboard slot 10
3,2,jk2_init() Found child 6726 in scoreboard slot 8
4,2,jk2_init() Found child 6728 in scoreboard slot 6
5,0,workerEnv.init() ok /etc/httpd/conf/workers2.p...
6,0,workerEnv.init() ok /etc/httpd/conf/workers2.p...
7,0,workerEnv.init() ok /etc/httpd/conf/workers2.p...
8,1,mod_jk child workerEnv in error state 6
9,1,mod_jk child workerEnv in error state 6


In [66]:
np.unique(predicted_labels)

array([0, 1, 2, 3, 4, 5])

In [67]:
clusters = {}
for i in range(0, len(predicted_labels)):
    try:
        clusters[predicted_labels[i]][i] = raw_logs[i].split(" ")
    except KeyError:
        clusters[predicted_labels[i]] = {i: raw_logs[i].split(" ")}





clusters



{0: {0: ['workerEnv.init()', 'ok', '/etc/httpd/conf/workers2.properties'],
  5: ['workerEnv.init()', 'ok', '/etc/httpd/conf/workers2.properties'],
  6: ['workerEnv.init()', 'ok', '/etc/httpd/conf/workers2.properties'],
  7: ['workerEnv.init()', 'ok', '/etc/httpd/conf/workers2.properties'],
  14: ['workerEnv.init()', 'ok', '/etc/httpd/conf/workers2.properties'],
  15: ['workerEnv.init()', 'ok', '/etc/httpd/conf/workers2.properties'],
  21: ['workerEnv.init()', 'ok', '/etc/httpd/conf/workers2.properties'],
  22: ['workerEnv.init()', 'ok', '/etc/httpd/conf/workers2.properties'],
  23: ['workerEnv.init()', 'ok', '/etc/httpd/conf/workers2.properties'],
  30: ['workerEnv.init()', 'ok', '/etc/httpd/conf/workers2.properties'],
  31: ['workerEnv.init()', 'ok', '/etc/httpd/conf/workers2.properties'],
  37: ['workerEnv.init()', 'ok', '/etc/httpd/conf/workers2.properties'],
  38: ['workerEnv.init()', 'ok', '/etc/httpd/conf/workers2.properties'],
  44: ['workerEnv.init()', 'ok', '/etc/httpd/conf/wo

In [83]:
output = {}
k = 0
lineIDS = []

for i in sorted(clusters.items()):
#     print(i)
#     print("+++++++++++++++++++++++++++++++")
    keys = list(i[1].keys())
#     print(keys)
    a = [list(v) for _, v in groupby(sorted(list(i[1].values()), key=len), key=len)]
#     print(a)
    for j in a:
        k += 1
        output['E' + str(k)] = getTemplate(j)

    l = [keys[len(j):], keys[:len(j)]]
    for c in l:
        if len(c) > 0:
            lineIDS += [c]
lineIDS

[[0,
  5,
  6,
  7,
  14,
  15,
  21,
  22,
  23,
  30,
  31,
  37,
  38,
  44,
  50,
  51,
  52,
  53,
  59,
  63,
  64,
  69,
  70,
  74,
  77,
  81,
  82,
  86,
  90,
  92,
  105,
  107,
  109,
  111,
  113,
  115,
  117,
  120,
  124,
  126,
  129,
  133,
  137,
  138,
  143,
  149,
  151,
  153,
  155,
  160,
  161,
  168,
  170,
  172,
  174,
  184,
  185,
  186,
  187,
  188,
  195,
  200,
  201,
  205,
  207,
  211,
  212,
  215,
  220,
  221,
  227,
  228,
  232,
  237,
  238,
  242,
  247,
  248,
  259,
  260,
  261,
  271,
  272,
  273,
  274,
  275,
  282,
  285,
  289,
  291,
  296,
  297,
  298,
  304,
  305,
  310,
  312,
  316,
  317,
  323,
  324,
  325,
  339,
  340,
  346,
  347,
  353,
  355,
  357,
  363,
  364,
  365,
  374,
  382,
  384,
  386,
  388,
  391,
  395,
  396,
  401,
  402,
  408,
  422,
  423,
  424,
  432,
  441,
  442,
  445,
  447,
  449,
  455,
  456,
  459,
  468,
  470,
  471,
  486,
  487,
  488,
  489,
  490,
  509,
  510,
  511,
  518,
  520

In [84]:
def writeOutput(output,lineno):
    line = {
        'EventId': list(output.keys()),
        'EventTemplate': list(output.values()),
        'LineNo': lineno
    }

    df = pd.DataFrame(line)
    df.to_csv(join_path(result_dir, experiment_dir, dataset + '_templates.csv'))


writeOutput(output,lineIDS)

In [93]:
structured_log = df_raw_logs.drop(['EventTemplate', 'EventId'], axis=1)
df_eventTemplates = pd.read_csv(join_path(result_dir, experiment_dir, dataset + '_templates.csv'),)

In [94]:
structured_log

,LineId,Time,Level,Content
0,1,Sun Dec 04 04:47:44 2005,notice,workerEnv.init() ok /etc/httpd/conf/workers2.p...
1,2,Sun Dec 04 04:47:44 2005,error,mod_jk child workerEnv in error state 6
2,3,Sun Dec 04 04:51:08 2005,notice,jk2_init() Found child 6725 in scoreboard slot 10
3,4,Sun Dec 04 04:51:09 2005,notice,jk2_init() Found child 6726 in scoreboard slot 8
4,5,Sun Dec 04 04:51:09 2005,notice,jk2_init() Found child 6728 in scoreboard slot 6
...,...,...,...,...
1995,1996,Mon Dec 05 19:14:11 2005,error,mod_jk child workerEnv in error state 6
1996,1997,Mon Dec 05 19:15:55 2005,notice,jk2_init() Found child 6791 in scoreboard slot 8
1997,1998,Mon Dec 05 19:15:55 2005,notice,jk2_init() Found child 6790 in scoreboard slot 7
1998,1999,Mon Dec 05 19:15:57 2005,notice,workerEnv.init() ok /etc/httpd/conf/workers2.p...


In [95]:
df_eventTemplates

,Unnamed: 0,EventId,EventTemplate,LineNo
0,0,E1,workerEnv.init() ok <*>,"[0, 5, 6, 7, 14, 15, 21, 22, 23, 30, 31, 37, 3..."
1,1,E2,mod_jk child workerEnv in error state <*>,"[1, 8, 9, 10, 16, 24, 25, 26, 32, 33, 39, 40, ..."
2,2,E3,jk2_init() Found child <*> in scoreboard slot <*>,"[2, 3, 4, 11, 12, 13, 17, 18, 19, 20, 27, 28, ..."
3,3,E4,[client <*>] Directory index forbidden by rule...,"[131, 579, 580, 581, 582, 583, 584, 585, 586, ..."
4,4,E5,mod_jk child init <*> <*>,"[795, 801, 1036, 1041, 1044, 1050, 1362, 1364,..."
5,5,E6,jk2_init() Can't find child <*> in scoreboard,"[784, 788, 1031, 1039, 1042, 1045, 1348, 1349,..."


In [107]:


# template = []
# eventId = []

#for label in predicted_labels:
#   template.append(df_eventTemplates[df_eventTemplates['EventId'] == 'E' + str(label)]['EventTemplate'])
#    eventId.append(df_eventTemplates[df_eventTemplates['EventId'] == 'E' + str(label)]['EventId'])

out = ['']*2000
print(len(lineIDS))

templates = list(output.values())

for i in range(0,len(lineIDS)):
    print('templates: ',templates[i])
    for j in lineIDS[i]:
        out[j] = templates[i]
    print('out', out)


# out

6
templates:  workerEnv.init() ok <*>
out ['workerEnv.init() ok <*>', '', '', '', '', 'workerEnv.init() ok <*>', 'workerEnv.init() ok <*>', 'workerEnv.init() ok <*>', '', '', '', '', '', '', 'workerEnv.init() ok <*>', 'workerEnv.init() ok <*>', '', '', '', '', '', 'workerEnv.init() ok <*>', 'workerEnv.init() ok <*>', 'workerEnv.init() ok <*>', '', '', '', '', '', '', 'workerEnv.init() ok <*>', 'workerEnv.init() ok <*>', '', '', '', '', '', 'workerEnv.init() ok <*>', 'workerEnv.init() ok <*>', '', '', '', '', '', 'workerEnv.init() ok <*>', '', '', '', '', '', 'workerEnv.init() ok <*>', 'workerEnv.init() ok <*>', 'workerEnv.init() ok <*>', 'workerEnv.init() ok <*>', '', '', '', '', '', 'workerEnv.init() ok <*>', '', '', '', 'workerEnv.init() ok <*>', 'workerEnv.init() ok <*>', '', '', '', '', 'workerEnv.init() ok <*>', 'workerEnv.init() ok <*>', '', '', '', 'workerEnv.init() ok <*>', '', '', 'workerEnv.init() ok <*>', '', '', '', 'workerEnv.init() ok <*>', 'workerEnv.init() ok <*>', '', 

In [108]:
#structured_log['EventId'] = eventId
structured_log['EventTemplate'] = out

structured_log.to_csv(join_path(result_dir, experiment_dir, dataset + '_structured.csv'))

In [109]:
def evaluate(groundtruth, parsedresult):
    df_groundtruth = pd.read_csv(groundtruth)
    df_parsedlog = pd.read_csv(parsedresult)

    # Remove invalid groundtruth event Templates
    null_logids = df_groundtruth[~df_groundtruth['EventTemplate'].isnull()].index
    df_groundtruth = df_groundtruth.loc[null_logids]
    df_parsedlog = df_parsedlog.loc[null_logids]

    (precision, recall, f_measure, accuracy) = get_accuracy(df_groundtruth['EventTemplate'],
                                                            df_parsedlog['EventTemplate'])
    print('Precision: %.4f, Recall: %.4f, F1_measure: %.4f, Parsing_Accuracy: %.4f' % (
        precision, recall, f_measure, accuracy))
    return f_measure, accuracy



In [110]:
def get_accuracy(series_groundtruth, series_parsedlog, debug=False):
    series_groundtruth_valuecounts = series_groundtruth.value_counts()
    real_pairs = 0
    for count in series_groundtruth_valuecounts:
        if count > 1:
            real_pairs += scipy.special.comb(count, 2)

    series_parsedlog_valuecounts = series_parsedlog.value_counts()
    parsed_pairs = 0
    for count in series_parsedlog_valuecounts:
        if count > 1:
            parsed_pairs += scipy.special.comb(count, 2)

    accurate_pairs = 0
    accurate_events = 0  # determine how many lines are correctly parsed
    for parsed_eventId in series_parsedlog_valuecounts.index:
        logIds = series_parsedlog[series_parsedlog == parsed_eventId].index
        series_groundtruth_logId_valuecounts = series_groundtruth[logIds].value_counts()
        error_eventIds = (parsed_eventId, series_groundtruth_logId_valuecounts.index.tolist())
        error = True
        if series_groundtruth_logId_valuecounts.size == 1:
            groundtruth_eventId = series_groundtruth_logId_valuecounts.index[0]
            if logIds.size == series_groundtruth[series_groundtruth == groundtruth_eventId].size:
                accurate_events += logIds.size
                error = False
        if error and debug:
            print('(parsed_eventId, groundtruth_eventId) =', error_eventIds, 'failed', logIds.size, 'messages')
        for count in series_groundtruth_logId_valuecounts:
            if count > 1:
                accurate_pairs += scipy.special.comb(count, 2)

    precision = float(accurate_pairs) / parsed_pairs
    recall = float(accurate_pairs) / real_pairs
    f_measure = 2 * precision * recall / (precision + recall)
    accuracy = float(accurate_events) / series_groundtruth.size
    return precision, recall, f_measure, accuracy


print(evaluate("gt/" + dataset + "_2k.log_structured.csv",  #####path of the structured log csv of the groundtruth
               result_dir + "/" + experiment_dir + "/" + dataset + '_structured.csv'))

os.system("rm -rf " + experiment_dir)

Precision: 1.0000, Recall: 1.0000, F1_measure: 1.0000, Parsing_Accuracy: 1.0000
(1.0, 1.0)


0